# アンサンブル検討用nb（XGboost、LightGBM、Catboost）×SVM

## Mount＆modjule import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 目的：スペインの電力価格を予測
# 目的変数：スペインの電力価格(actual_price)
# 評価指標：RME

import numpy as np
import pandas as pd
import os
import pickle
import gc
# 分布確認
!pip install ydata-profiling
from ydata_profiling import ProfileReport
# 可視化
import matplotlib.pyplot as plt
import seaborn as sns
import polars as pl
# 前処理
from sklearn.preprocessing import StandardScaler, MinMaxScaler,LabelEncoder,OneHotEncoder
# モデリング
from sklearn.model_selection import train_test_split,KFold,StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from xgboost import XGBClassifier
import lightgbm as lgb
import xgboost as xgb
!pip install -U xgboost -q
!pip install -q catboost
from catboost import CatBoostRegressor, Pool

# 日本語表記
!pip install japanize-matplotlib
import japanize_matplotlib
%matplotlib inline
# パラメータ最適化
!pip install optuna
import optuna

# 評価指標
from sklearn.metrics import mean_squared_error

from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
import holidays
from collections import deque


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.1/400.1 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.0/679.0 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 2.5 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=4a4a3667ebf0b9679a8e38fcad1632d26ea44b9a131d9a62941e4a04b23a159b
  Stored in directory: /root/.cache/pip/wheels/8d/55/1a/19cd535375ed1ede0c996405ebffe34b196d78e2d9545723a2
Successfully built htmlmin
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.0
    Uninstalling scipy-1.16.0:
     

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 32.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120257 sha256=3d01fd9538fcb835139a6ac4c10633d06ac792913edff4451b7c5c15b227cbac
  Stored in directory: /root/.cache/pip/wheels/da/a1/71/b8faeb93276fed10edffcca20746f1ef6f8d9e071eee8425fc
Successfully built japanize-matplotlib
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 14.7 MB/s eta 0:00:00


In [3]:
params_xgb = {
    "objective":       "reg:squarederror",
    "learning_rate":   0.023378586570803068,
    "max_depth":       7,
    "n_estimators":    317,
    "subsample":       0.9665142871966073,
    "colsample_bytree":0.6519917349250606,
    "min_child_weight":1,
    "gamma":           0.02005153624696357,
    "lambda":          0.6361532102889825,  # L2 正則化
    "tree_method":     "hist",
    "random_state":    42,
    "eval_metric":     "rmse"
}


In [4]:
dir_path = '/content/drive/MyDrive/Colab Notebooks/signate/smbc/'

train = pd.read_csv(dir_path + 'train.csv')
test  = pd.read_csv(dir_path + 'test.csv')

## 前処理

In [5]:
# -*- coding: utf-8 -*-
"""
Integrated recursive forecasting pipeline:
 - Preprocessing & feature engineering
 - PCA + SVR base model
 - XGBoost base model
 - Ridge meta-model stacking
 - Recursive forecasting on test set
"""
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
import xgboost as xgb
import holidays
from collections import deque

# 1. Load data
dir_path = '/content/drive/MyDrive/Colab Notebooks/signate/smbc/'  # adjust path
train = pd.read_csv(dir_path + 'train.csv')
test  = pd.read_csv(dir_path + 'test.csv')

# 2. Preprocessing & feature engineering
def preprocess(df, is_train=False):
    # time features
    df['time']    = pd.to_datetime(df['time'], utc=True)
    df['year']    = df['time'].dt.year
    df['month']   = df['time'].dt.month
    df['day']     = df['time'].dt.day
    df['hour']    = df['time'].dt.hour
    df['weekday'] = df['time'].dt.weekday

    # season
    df['season'] = df['month'].map(
        lambda m: 1 if 3 <= m <= 5 else
                  2 if 6 <= m <= 8 else
                  3 if 9 <= m <= 11 else
                  4
    )

    # time_of_day & weekend
    df['time_of_day'] = df['hour'].map(
        lambda h: 1 if 5 <= h < 11 else
                  2 if 11 <= h < 17 else
                  3 if 17 <= h < 21 else
                  4
    )
    df['is_weekend'] = df['weekday'].map(lambda w: 1 if w >= 5 else 0)

    # temperature (Kelvin→℃)
    cities    = ["valencia","madrid","bilbao","barcelona","seville"]
    temp_cols = [f"{c}_{t}" for c in cities for t in ["temp","temp_min","temp_max"]]
    for c in temp_cols:
        df[c] = df[c] - 273.15
    df['valencia_temp_diff'] = df['valencia_temp_max'] - df['valencia_temp_min']
    df['temp_dev']           = (df['valencia_temp'] - 23.0).abs()

    # generation sums & shares
    gen_cols = [c for c in df.columns if c.startswith('generation_')]
    df['generation_sum'] = df[gen_cols].sum(axis=1)
    df['fossil_total']   = (df['generation_fossil_gas']
                           + df['generation_fossil_hard_coal']
                           + df['generation_fossil_oil'])
    df['renewable_total'] = (
        df['generation_hydro_pumped_storage_consumption']
      + df['generation_hydro_run_of_river_and_poundage']
      + df['generation_hydro_water_reservoir']
      + df['generation_other_renewable']
    )
    eps = 1e-6
    df['fossil_share']    = df['fossil_total'] / (df['generation_sum'] + eps)
    df['renewable_share'] = df['renewable_total'] / (df['generation_sum'] + eps)

    # share diffs
    df['fossil_share_diff1']    = df['fossil_share'] - df['fossil_share'].shift(1)
    df['renewable_share_diff1'] = df['renewable_share'] - df['renewable_share'].shift(1)

    # lags for loads and generation_sum
    df.sort_values('time', inplace=True)
    for lag in (1, 24, 168):
        df[f'total_load_actual_lag{lag}'] = df['total_load_actual'].shift(lag)
        df[f'generation_sum_lag{lag}']    = df['generation_sum'].shift(lag)

    # train‐only lags for price_actual
    if is_train:
        for lag in (1, 24, 168):
            df[f'price_actual_lag{lag}'] = df['price_actual'].shift(lag)

    # pct change
    df['total_load_actual_pct1h'] = df['total_load_actual'].pct_change()
    df['generation_sum_pct1h']    = df['generation_sum'].pct_change()

    # cyclical hour
    theta       = 2 * np.pi * df['hour'] / 24
    df['hour_sin'], df['hour_cos'] = np.sin(theta), np.cos(theta)

    # interactions
    df['load_sin']   = df['hour_sin'] * df['total_load_actual']
    df['load_cos']   = df['hour_cos'] * df['total_load_actual']
    df['load_x_tod'] = df['total_load_actual'] * df['time_of_day']

    # holidays
    es = holidays.Spain(years=[2015,2016,2017,2018])
    df['is_holiday'] = df['time'].dt.date.map(lambda d: 1 if d in es else 0)
    for s in (-1, +1):
        df[f'hol_adj{s}'] = df['is_holiday'].shift(s).fillna(0)
    df['is_long_wend'] = (df['is_holiday']
                          .rolling(3, min_periods=1)
                          .sum() >= 2).astype(int)

    # clean up numeric columns only
    num_cols = df.select_dtypes(include=[np.number]).columns
    df[num_cols] = df[num_cols].replace([np.inf, -np.inf], np.nan)
    df[num_cols] = df[num_cols].fillna(df[num_cols].median())
    df[num_cols] = df[num_cols].clip(-1e6, 1e6)

    return df

# 3. Apply preprocessing
train = preprocess(train, is_train=True)
test  = preprocess(test,  is_train=False)

# 4. Fill test's price_actual lags from train
train_price = train.set_index('time')['price_actual']
for lag in (1, 24, 168):
    test[f'price_actual_lag{lag}'] = test['time'].apply(
        lambda t: train_price.get(t - pd.Timedelta(hours=lag), np.nan)
    )
# clean again
num_t = test.select_dtypes(include=[np.number]).columns
test[num_t] = test[num_t].fillna(test[num_t].median())
test[num_t] = test[num_t].clip(-1e6, 1e6)

# 5. Define feature list
base_features = [
    'generation_fossil_gas','generation_fossil_hard_coal',
    'generation_hydro_pumped_storage_consumption',
    'generation_hydro_run_of_river_and_poundage',
    'generation_hydro_water_reservoir','generation_other_renewable',
    'total_load_actual'
]
add_features = [
    'generation_biomass','generation_fossil_brown_coal/lignite',
    'madrid_wind_speed','bilbao_pressure','bilbao_wind_speed',
    'barcelona_pressure','barcelona_wind_speed',
    'seville_pressure','seville_wind_deg',
    'time_of_day','is_weekend',
    'valencia_temp_diff','temp_dev',
    'fossil_share_diff1','renewable_share_diff1',
    'total_load_actual_pct1h','generation_sum_pct1h',
    'hour_sin','hour_cos','load_sin','load_cos','load_x_tod',
    'price_actual_lag1','price_actual_lag24',
    'total_load_actual_lag1','total_load_actual_lag24',
    'generation_sum_lag1','generation_sum_lag24'
]
features = base_features + add_features

# 6. OOF stacking with PCA-SVM & XGB
FOLDS         = 4
tscv          = TimeSeriesSplit(n_splits=FOLDS)
oof_xgb       = np.zeros(len(train))
oof_svm       = np.zeros(len(train))
test_pred_xgb = np.zeros(len(test))
test_pred_svm = np.zeros(len(test))

# prepare PCA for SVM
pca_svm = PCA(n_components=10, random_state=42)
pca_svm.fit(train[features])

for fold, (tr, va) in enumerate(tscv.split(train)):
    X_tr, X_va = train.iloc[tr][features], train.iloc[va][features]
    y_tr, y_va = train.iloc[tr]['price_actual'], train.iloc[va]['price_actual']

    # XGB
    xgb_model = xgb.XGBRegressor(**params_xgb).fit(X_tr, y_tr)
    oof_xgb[va]        = xgb_model.predict(X_va)
    test_pred_xgb     += xgb_model.predict(test[features]) / FOLDS

    # PCA-SVM
    svm_pipe = make_pipeline(
        MinMaxScaler(),
        pca_svm,
        SVR(kernel='rbf', C=3.0, gamma='auto')
    )
    svm_pipe.fit(X_tr, y_tr)
    oof_svm[va]        = svm_pipe.predict(X_va)
    test_pred_svm     += svm_pipe.predict(test[features]) / FOLDS


# meta-model 学習用データ は OOF 同士を並べる
X_meta = np.column_stack([oof_xgb, oof_svm])
y_meta = train['price_actual'].values

# ─────────────────────────────────────
# 6.5. Fit the meta‐model (Ridge)
# ─────────────────────────────────────
from sklearn.linear_model import Ridge
meta = Ridge(alpha=1.0, random_state=42)
meta.fit(X_meta, y_meta)

# テスト時のメタ入力はテスト予測同士
meta_X_test = np.column_stack([test_pred_xgb, test_pred_svm])

# full retrain
xgb_full = xgb.XGBRegressor(**params_xgb).fit(train[features], train['price_actual'])
svm_full = make_pipeline(MinMaxScaler(), pca_svm,
                          SVR(kernel='rbf', C=3.0, gamma='auto')
                ).fit(train[features], train['price_actual'])

# 7. Recursive forecasting
test_sorted = test.sort_values('time').reset_index(drop=True)
last_price  = train['price_actual'].iloc[-1]
buf24       = deque(train['price_actual'].tail(24), maxlen=24)

final_preds = []
for _, row in test_sorted.iterrows():
    # update lags
    row['price_actual_lag1']  = last_price
    row['price_actual_lag24'] = buf24[-1]
    X_row = row[features].values.reshape(1, -1)

    p1 = xgb_full.predict(X_row)[0]
    p2 = svm_full.predict(X_row)[0]
    p_hat = meta.predict(np.array([[p1, p2]]))[0]

    final_preds.append(p_hat)
    last_price = p_hat
    buf24.append(p_hat)

# 8. Save submission
out = pd.read_csv(dir_path + 'sample_submit.csv', header=None, names=['time','pred'])
out['pred'] = final_preds
out.to_csv('submission_recursive_stack.csv', index=False, header=False)

print("✅ Done. submission_recursive_stack.csv created.")


ストリーミング出力は最後の 5000 行に切り捨てられました。
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/di

✅ Done. submission_recursive_stack.csv created.


In [ ]:
# 6) 提出ファイル作成
df_sub = pd.read_csv(file_path + 'sample_submit.csv', header=None)
submission = pd.DataFrame({
    'time': df_sub.iloc[:, 0],
    'pred': preds_recursive    # 8760個の予測値
})
submission.to_csv("submission_recursive.csv", index=False, header=False)